In [5]:
import subprocess
import numpy as np
import sys

if sys.platform=='linux':
    indir='./Linux_results/'
else:
    indir='.\\win_results\\'

np.set_printoptions(precision=15)

def Ising(ell: int, gfield: np.double, hfield:np.double, PBC: bool, parity: bool):
    if parity:
        if hfield!=0.:
            print("WARNING: hfield has to be zero in order to implement the parity simmetry")
        fileout='Ising_Parity'
    else:
        fileout='Ising'
        
    f=open("chain.in", "w")
    f.write(
f"{ell}	    ! Number of spins in the system		(ell)\n\
{gfield}	! Transverse magnetic field strength	(g)\n\
{hfield}	! Longitudinal magnetic field strength	(h)\n\
.{'true' if PBC else 'false'}.	! Type of boundary conditions  		(.true. -> PBC,        .false. -> OBC)\n\
.true.	! Type of diagonalization      		(.true. -> Davidson,   .false. -> Lapack full diag) NOT IMPLEMENTED\n"\
        )
    f.close()
    p=subprocess.Popen(indir+fileout, shell=True, stdout=subprocess.PIPE)
    out=p.communicate()[0].decode().replace('\n','').split(sep=',')
    out_ell=float(out.pop(0))
    out_gfield=float(out.pop(0))
    if parity:
        E_p=np.array([np.double(out.pop(0)) for _ in range(3)],dtype=np.double)
        E_m=np.array([np.double(out.pop(0)) for _ in range(3)],dtype=np.double)
        mag=abs(np.double(out.pop(0)))
        return E_p, E_m, mag        
    E=np.array([np.double(out.pop(0)) for _ in range(3)])
    broken_mag=np.abs(np.double(out.pop(0)))
    if PBC:
        long_mag=np.array([np.double(out.pop(0)) for _ in range(ell)])
        tran_mag=np.array([np.double(out.pop(0)) for _ in range(ell)])
    else:
        long_mag=np.double(out.pop(0))
        tran_mag=np.double(out.pop(0))
    return E, broken_mag, long_mag, tran_mag

""" def Ising_Parity(ell: int, gfield: float, PBC: bool):
    f=open("chain.in", "w")
    f.write(
f"{ell}	    ! Number of spins in the system		(ell)\n\
{gfield}d0	! Transverse magnetic field strength	(g)\n\
0.d0	! Longitudinal magnetic field strength	(h)\n\
.{'true' if PBC else 'false'}.	! Type of boundary conditions  		(.true. -> PBC,        .false. -> OBC)\n\
.true.	! Type of diagonalization      		(.true. -> Davidson,   .false. -> Lapack full diag) NOT IMPLEMENTED\n"\
        )
    f.close()
    p=subprocess.Popen(indir+"Ising_Parity", shell=True, stdout=subprocess.PIPE)
    out=p.communicate()[0].decode().replace('\n','').split(sep=',')
    out_ell=float(out.pop(0))
    out_gfield=float(out.pop(0))
    E_p=np.array([float(out.pop(0)) for _ in range(3)])
    E_m=np.array([float(out.pop(0)) for _ in range(3)])
    mag=abs(float(out.pop(0)))
 """

if __name__=='__main__':
    print(Ising(15,0.2, 0.5, False, parity=False))







import os
def automatic_sampler (Ls, gfields, hfields, PBC: bool, parity:bool):
    #Idea: Ls, gfields, hfields can also be a single number, rather than an one-dimensional array. In that case, that variable is fixed.
    
    OUTPUTFILE="computed_stuff.out"
    if os.path.isfile(OUTPUTFILE):
        fout=open(OUTPUTFILE,"a")
    else:
        fout=open(OUTPUTFILE,"w")
        fout.write("L,g,h,E_ground,long_mag,trans_mag\n") #Qui ho deciso di mettere solo queste variabili e solo in questo ordine, poi giustamente se tu vuoi cambiarlo o se vuoi aggiungere altre energie oltre a quella di ground ne parliamo e possiamo modificare questa funzione di prova
    
    for L in Ls:
        for gfield in gfields:
            for hfield in hfields:
                Es, broken_mag, long_mag, trans_mag=Ising(L, gfield, hfield, PBC, parity)
                E_ground=Es[0]
                
                newrow= f"{L},{gfield},{hfield},{E_ground},{long_mag},{trans_mag}\n"
                fout.write(newrow)
    

    fout.close()
    return 0

(array([-21.6306998432344, -18.6359672411513, -18.6359672411513]), 0.996025179973269, 0.991139690362512, 0.99674032273216)


In [10]:
Ls=[15]
gfields=np.linspace(0.05,0.95,10)
hfields=np.linspace(-0.01,+0.01,20)

automatic_sampler(Ls,gfields,hfields,False,False)

0